<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/joao/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [ ]:
!pip install dotenv

## Importações

In [ ]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
import json
from dotenv import load_dotenv
from google.colab import userdata
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

## Pegando as chaves

In [ ]:
# load_dotenv()
# PUBLIC_KEY = os.getenv("Public_Key")
# PRIVATE_KEY = os.getenv("Private_Key")

PRIVATE_KEY = userdata.get("Private_Key")
PUBLIC_KEY = userdata.get("Public_Key")


## Endpoints e Parâmetros

In [ ]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

endCharacters = "https://gateway.marvel.com/v1/public/characters"
endComics = "https://gateway.marvel.com/v1/public/comics"
endSeries = "https://gateway.marvel.com/v1/public/series"
endEvents = "https://gateway.marvel.com/v1/public/events"
endCreators = "https://gateway.marvel.com/v1/public/creators"


params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 10
}


## Pegando o total de requisições

In [ ]:
totalCharacters = requests.get(endCharacters, params=params)
totalCharacters = totalCharacters.json()
total = totalCharacters['data']['total']

totalEvents = requests.get(endEvents, params=params)
totalEvents = totalEvents.json()
totalE = totalEvents['data']['total']

totalCreators = requests.get(endCreators, params=params)
totalCreators = totalCreators.json()
totalC = totalCreators['data']['total']

## Função para realizar requisições

In [ ]:

def requisition(endpoint, total, step, filename, limit, params_base=None):
    data_array = []
    offset = 0

    if params_base is None:
        params_base = {}

    while offset <= total:
        params = params_base.copy()
        params['offset'] = offset
        params['limit'] = limit

        response = requests.get(endpoint, params=params)

        if response.status_code != 200:
            print(f"Erro no offset {offset}: status code {response.status_code}")
            offset += step
            continue

        try:
            data = response.json()
        except ValueError:
            print(f"Erro ao converter JSON no offset {offset}")
            offset += step
            continue

        if not data or 'data' not in data or 'results' not in data['data']:
            print(f"Resposta inesperada ou vazia no offset {offset}")
            offset += step
            continue

        data_array.extend(data['data']['results'])
        print(f"Offset: {offset} | Status: {response.status_code}")
        offset += step

    df = pd.DataFrame(data_array)
    df.to_csv(filename, index=False)
    print(f"Dados salvos em {filename}")
    return df

## Chamando as requisições

In [ ]:
df_characters = requisition(endCharacters, total, step=100, filename='Characters.csv', limit=100, params_base=params)

# Events
df_events = requisition(endEvents, totalE, step=10, filename='Events.csv', limit=10, params_base=params)

# Creators
df_creators = requisition(endCreators, totalC, step=100, filename='Creators.csv', limit=100,  params_base=params)

## Realizando Converções

In [ ]:
df_characters = df_characters[['name', 'id', 'description', 'comics', 'thumbnail']]
df_events = df_events[['title', 'id', 'description', 'characters', 'creators', 'comics', 'start', 'end', 'modified']]
df_creators = df_creators[['id', 'firstName', 'middleName', 'lastName', 'fullName', 'suffix', 'thumbnail','comics', 'events', 'stories', 'series']]
df_characters['comics_available'] = df_characters['comics'].apply(lambda x: x.get('available'))
df_characters['comics_returned'] = df_characters['comics'].apply(lambda x: x.get('returned'))
df_characters['thumbnail_path'] = df_characters['thumbnail'].apply(lambda x: x.get('path'))
df_characters['comics'] = df_characters['comics'].apply(lambda x: json.dumps(x))
df_characters['thumbnail'] = df_characters['thumbnail'].apply(lambda x: json.dumps(x))
df_events['comics_available'] = df_events['comics'].apply(lambda x: x.get('available'))
df_events['comics_returned'] = df_events['comics'].apply(lambda x: x.get('returned'))
df_events['comics'] = df_events['comics'].apply(lambda x: json.dumps(x))
df_events['creators_available'] = df_events['creators'].apply(lambda x: x.get('available'))
df_events['creators'] = df_events['creators'].apply(lambda x: json.dumps(x))
df_events['characters_available'] = df_events['characters'].apply(lambda x: x.get('available'))
df_events['characters'] = df_events['characters'].apply(lambda x: json.dumps(x))
df_creators['comics_available'] = df_creators['comics'].apply(lambda x: x.get('available'))
df_creators['events_available'] = df_creators['events'].apply(lambda x: x.get('available'))
df_creators['stories_available'] = df_creators['stories'].apply(lambda x: x.get('available'))
df_creators['series_available'] = df_creators['series'].apply(lambda x: x.get('available'))
df_creators['thumbnail_path'] = df_creators['thumbnail'].apply(lambda x: x.get('path'))
df_creators['comics'] = df_creators['comics'].apply(lambda x: json.dumps(x))
df_creators['events'] = df_creators['events'].apply(lambda x: json.dumps(x))
df_creators['series'] = df_creators['series'].apply(lambda x: json.dumps(x))
df_creators['stories'] = df_creators['stories'].apply(lambda x: json.dumps(x))
df_creators['thumbnail'] = df_creators['thumbnail'].apply(lambda x: json.dumps(x))

## Salvando no Banco de Dados

In [ ]:
con = sqlite3.connect('Marvel.db')
df_characters.to_sql("characters", con, if_exists="replace", index=False)
df_events.to_sql("events", con, if_exists="replace", index=False)
df_creators.to_sql("creators", con, if_exists="replace", index=False)
con.close()

## Consultando o Banco de Dados

In [ ]:
con2 = sqlite3.connect('Marvel.db')

df2 = pd.read_sql_query("SELECT id, name, description, comics, comics_available, comics_returned FROM characters", con2)

query1 = """
SELECT SUBSTR(name, 1, 1) AS letra_inicial, COUNT(*) AS total_por_letra
FROM characters
GROUP BY letra_inicial
ORDER BY letra_inicial;
"""
query2 = """
SELECT id, name, comics_available
FROM characters
ORDER BY comics_available DESC
LIMIT 10;
"""

query3 = """
SELECT id, name, description, comics_available
FROM characters
WHERE description IS NOT NULL AND description != '';
"""

query3_1 = """
SELECT
  COUNT (*) FILTER (WHERE description IS NOT NULL AND description != '') AS com_descricao,
  COUNT (*) FILTER (WHERE description IS NULL OR description = '') AS sem_descricao
FROM characters;
"""

query4 = """
SELECT title, comics_available
FROM events
ORDER BY comics_available DESC
LIMIT 10;
"""

query5 = """
SELECT title,
       strftime('%d/%m/%Y', start) AS start_formatted,
       strftime('%d/%m/%Y', end) AS end_formatted,
       CAST((julianday(end) - julianday(start)) / 365.25 AS INTEGER) AS duration_in_years,
       comics_available
FROM events
WHERE julianday(start) <= julianday('2025-05-16') AND julianday(end) <= julianday('2025-05-16')
ORDER BY julianday(end) - julianday(start) DESC;
"""

query6 = """
SELECT title,
       strftime('%d/%m/%Y', SUBSTR(modified, 1, 10)) AS modified,
       strftime('%d/%m/%Y', start) AS start,
       strftime('%d/%m/%Y', end) AS end
FROM events
ORDER BY SUBSTR(modified, 1, 4) DESC;
"""

query7 = """
SELECT title, characters_available, comics_available
FROM events
ORDER BY characters_available DESC
LIMIT 10;
"""

query8 = """
SELECT title, characters_available, creators_available, comics_available
FROM events
WHERE characters_available > 0 AND creators_available > 0 AND comics_available > 0;
"""


query9 = """
SELECT  firstName, middleName, lastName, fullName, comics_available, events_available
FROM creators
ORDER BY comics_available DESC
LIMIT 15;
"""

query10 = """
SELECT
  COUNT(*) FILTER (WHERE thumbnail_path NOT LIKE '%image_not_available') AS Criadores_Com_Imagem_Associada,
  COUNT(*) FILTER (WHERE thumbnail_path LIKE '%image_not_available') AS Criadores_Sem_Imagem_Associada
FROM creators;
"""

query11 = """
SELECT
  COUNT(*) FILTER (WHERE thumbnail_path NOT LIKE '%image_not_available') AS Personagens_Com_Imagem_Associada,
  COUNT(*) FILTER (WHERE thumbnail_path LIKE '%image_not_available') AS Personagens_Sem_Imagem_Associada
FROM characters;
"""

count_letters = pd.read_sql_query(query1, con2)
most_comics_charatcers = pd.read_sql_query(query2, con2)
description_characters = pd.read_sql_query(query3, con2)
most_events_comics = pd.read_sql_query(query4, con2)
most_duration_events = pd.read_sql_query(query5, con2)
events_modified = pd.read_sql_query(query6, con2)
events_most_characters = pd.read_sql_query(query7, con2)
count_description = pd.read_sql_query(query3_1, con2)
comics_per_events = pd.read_sql_query(query9, con2)
creators_images = pd.read_sql_query(query10, con2)
characters_images = pd.read_sql_query(query11, con2)

con2.close()

# Funções Auxiliares

In [ ]:
def plot_pie_charts(df, colors=None, figsize=(10, 6)):
    # Plotar os gráficos de pizza
    df_transposed = df.transpose()

    axes = df_transposed.plot.pie(
        subplots=True,
        autopct='%1.1f%%',
        figsize=figsize,
        legend=False,
        startangle=90,
        colors=colors
    )

    # Títulos e ajustes
    for ax, col in zip(axes, df_transposed.columns):
        ax.set_ylabel('')  # Remove rótulo do eixo Y
        ax.set_title(col, fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()

def verify_first_letters(nome_completo):
  partes_nome = nome_completo.split()
  if len(partes_nome) >= 2:
    nome = partes_nome[0]
    sobrenome = partes_nome[-1] # Pega a última parte como sobrenome
    if nome and sobrenome: # Verifica se nome e sobrenome não são vazios
      return nome[0].lower() == sobrenome[0].lower()
  return False # Retorna False se não for possível verificar (nome curto ou vazio)

def plot_side_by_side_pies(series1, series2, titles=None, colors=None, figsize=(12, 6)):
    fig, axes = plt.subplots(1, 2, figsize=figsize)

    series1.plot.pie(
        ax=axes[0],
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        legend=False
    )

    series2.plot.pie(
        ax=axes[1],
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        legend=False
    )

    if titles:
        axes[0].set_title(titles[0], fontsize=12, fontweight='bold')
        axes[1].set_title(titles[1], fontsize=12, fontweight='bold')

    axes[0].set_ylabel('')
    axes[1].set_ylabel('')

    plt.tight_layout()
    plt.show()

# Characters

## INSIGHT 1 Characters - Criando função para selecionar entidades que tenham nome aliterativo

In [ ]:
# Aplica a função à coluna 'name' e cria uma nova coluna booleana
df_characters['nome_sobrenome'] = df2['name'].apply(verify_first_letters)

# Para ver os personagens que atendem à condição
df_same_letter = df_characters[df_characters['nome_sobrenome']]
df_same_letter[['name', 'nome_sobrenome']]

## INSIGHT 2 Characters - Número de entidades por letra

In [ ]:
count_letters

## INSIGHT 2 Characters - Gráfico

In [ ]:
count_letters.plot.bar(x='letra_inicial', y='total_por_letra', legend=False)

## INSIGHT 3 Characters - Demonstração dos 10 personagens que tem mais quadrinhos

In [ ]:
most_comics_charatcers.plot.bar(x='name', y='comics_available')

## INSIGHT 4 Characters - Tabela e Gráfico comparativo de Personagens que tem descrição

In [ ]:
description_characters

In [ ]:
plot_pie_charts(count_description)

## INSIGHT 5 Characters - Gráfico de personagens que tem imagens e uma comparação a quantidade de Persongagens que tem descrição

In [ ]:
plot_pie_charts(characters_images)

In [ ]:
plot_side_by_side_pies(
    count_description.iloc[0],
    characters_images.iloc[0],
    titles=["Distribuição de Descrição", "Distribuição de Imagem"],
    colors=['#66c2a5', '#fc8d62']
)

# Events

## INSIGHT 1 Events - Os 10 eventos com mais histórias em quadrinhos

In [ ]:
most_events_comics.plot.bar(x='title', y='comics_available')

## INSIGHT 2 Events - Os eventos com maior duração

In [ ]:
most_duration_events

In [ ]:
#df9
df_timeline = events_modified.copy()

# Converter datas de string para datetime
df_timeline['start'] = pd.to_datetime(df_timeline['start'], format='%d/%m/%Y', errors='coerce')
df_timeline['end'] = pd.to_datetime(df_timeline['end'], format='%d/%m/%Y', errors='coerce')

# Remover eventos com datas inválidas
df_timeline = df_timeline.dropna(subset=['start', 'end'])

# Ordenar por data de início
df_timeline = df_timeline.sort_values(by='start')
plt.figure(figsize=(12, len(df_timeline) * 0.14))  # ajusta altura dinamicamente
plt.margins(y=0)

# Índice para os nomes dos eventos
y_positions = range(len(df_timeline))

# Plotar as barras horizontais
plt.barh(
    y=y_positions,
    width=(df_timeline['end'] - df_timeline['start']).dt.days,
    left=df_timeline['start'],
    color='skyblue'
)

# Nome dos eventos no eixo Y
plt.yticks(ticks=y_positions, labels=df_timeline['title'])

# Formatando o eixo X para mostrar ano
plt.gca().xaxis_date()
plt.xlabel('Ano')
plt.title('Linha do Tempo dos Eventos Marvel')
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## INSIGHT 3 Events - Eventos modificados mais recentemente

In [ ]:
events_modified

## INSIGHT 4 Events - Distribuição de Personagens por Duração do Evento dos 10 Eventos mais longos

In [ ]:
plt.figure(figsize=(10, 6))  # ajusta altura dinamicamente
scatter = plt.scatter(
    x=events_most_characters['characters_available'],
    y=events_most_characters['title'],
    s=events_most_characters['comics_available'] * 2,  # tamanho da bolha (ajuste o fator)
    c=events_most_characters['comics_available'],  # cor baseada em quadrinhos
    cmap='viridis',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Personagens Disponíveis')
plt.ylabel('Titulo do Evento')
plt.title('Escala dos Eventos Marvel: Relação entre Personagens e Quadrinhos')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

# Creators

## INSIGHT 1 Creators - Distribuição entre quantos comics o autor escreveu e em quantos eventos estava

In [ ]:
plt.figure(figsize=(10, 8))

scatter = plt.scatter(
    x=comics_per_events['events_available'],
    y=comics_per_events['fullName'],
    s=comics_per_events['comics_available'],  # tamanho da bolha
    c=comics_per_events['comics_available'],      # cor baseada nos quadrinhos
    cmap='plasma',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Eventos Disponíveis')
plt.ylabel('Nome do Criador')
plt.title('Criadores: Participação em Eventos vs Quadrinhos (Tamanho/Cor)')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

## INSIGHT 2 Creators - Comparação de Creators com imagens associadas e sem imagens associadas

In [ ]:
plot_pie_charts(creators_images)